<div style='text-align: center;'>
<h6>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<br>    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    </h6>
    <b><h1><u>
        Customer complaints analysis
        </b></h1></u>
<h6>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<br>    
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    </h6>
    </div>


---
## Importing Python libraries
---

In [43]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 2000
import time
import emoji

from collections import defaultdict

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD

!python -m pip install -U pip setuptools wheel
!python -m pip install -U spacy
!python -m spacy download en_core_web_sm
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

import gensim
from gensim.models import HdpModel

!python -m pip install tomotopy
import tomotopy as tp

from IPython.display import Markdown as md
!python -m pip install jupyter_contrib_nbextensions && jupyter contrib nbextension install --user
!jupyter nbextension enable python-markdown/main

     ---------------------------------------- 12.8/12.8 MB 2.1 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


[I 23:07:27 InstallContribNbextensionsApp] jupyter contrib nbextension install --user
[I 23:07:27 InstallContribNbextensionsApp] Installing jupyter_contrib_nbextensions nbextension files to jupyter data directory
[I 23:07:27 InstallContribNbextensionsApp] Installing C:\Apps\Anaconda\lib\site-packages\jupyter_contrib_nbextensions\nbextensions\addbefore -> addbefore
[I 23:07:27 InstallContribNbextensionsApp] Up to date: C:\Users\abeer\AppData\Roaming\jupyter\nbextensions\addbefore\addbefore.yaml
[I 23:07:27 InstallContribNbextensionsApp] Up to date: C:\Users\abeer\AppData\Roaming\jupyter\nbextensions\addbefore\icon.png
[I 23:07:27 InstallContribNbextensionsApp] Up to date: C:\Users\abeer\AppData\Roaming\jupyter\nbextensions\addbefore\main.js
[I 23:07:27 InstallContribNbextensionsApp] Up to date: C:\Users\abeer\AppData\Roaming\jupyter\nbextensions\addbefore\readme.md
[I 23:07:27 InstallContribNbextensionsApp] - Validating: ok
[I 23:07:27 InstallContribNbextensionsApp] Installing C:\Apps\A

Enabling notebook extension python-markdown/main...
      - Validating: ok


---
## Importing complaints data from the CSV file
---

<div style='text-align: left;'>
    <h3><i><u>
        Arizona complaints text 
        </h3></i></u>
</div>


In [2]:
# Importing the 'complaints_text_arizona' column from the arizona_massachusetts_comcast_complaints CSV file containing the 
# complaints text from Arizona

customer_complaints_arizona = pd.read_csv("arizona_massachusetts_comcast_complaints.csv", 
                                          usecols=["complaints_text_arizona"])

In [3]:
# Displaying the first five rows of the customer complaints from Arizona

customer_complaints_arizona.head()

,complaints_text_arizona
0,"First off, I've been without internet for a we..."
1,"So, after having gone over Comcast's measly 30..."
2,After 8 years I was very disappointed with my ...
3,This is one company you do not want to deal wi...
4,"Okay, I had this technician come out about 2 w..."


In [4]:
# Total number of complaints from Arizona

print("................................................................................\n",
      "Total number of rows from Arizona is: ", len(customer_complaints_arizona), 
      "\n................................................................................\n")

# Filtering the complaints data to display only the rows containing a complaint and removing any empty/null rows

customer_complaints_arizona=customer_complaints_arizona[pd.notnull(customer_complaints_arizona['complaints_text_arizona'])]
number_of_non_null_rows = customer_complaints_arizona.shape[0]
print("The total number of rows after removing null entries is: ", str(number_of_non_null_rows), 
      "\n................................................................................\n")
print(customer_complaints_arizona[slice(5)])    # Displaying the first 5 rows for Arizona

................................................................................
 Total number of rows from Arizona is:  138 
................................................................................

The total number of rows after removing null entries is:  17 
................................................................................

                             complaints_text_arizona
0  First off, I've been without internet for a we...
1  So, after having gone over Comcast's measly 30...
2  After 8 years I was very disappointed with my ...
3  This is one company you do not want to deal wi...
4  Okay, I had this technician come out about 2 w...


<div style='text-align: left;'>
    <h3><i><u>
        Massachusetts complaints text 
        </h3></i></u>
</div>


In [5]:
# Importing the 'complaints_text_massachusetts' column from the arizona_massachusetts_comcast_complaints CSV file 
# containing the complaints text from Massachusetts

customer_complaints_massachusetts = pd.read_csv("arizona_massachusetts_comcast_complaints.csv", 
                                          usecols=["complaints_text_massachusetts"])

In [6]:
# Displaying the first five rows of the customer complaints from Massachusetts

customer_complaints_massachusetts.head()

,complaints_text_massachusetts
0,Had Comcast. Overall the terrible experience e...
1,I was experiencing some technical difficulties...
2,"Consistently, with the last couple of weeks (a..."
3,Been dealing with a nightmare at Comcast for a...
4,I signed up for a one day pass with XFINITY wh...


In [7]:
# Total number of complaints from Massachusetts

print("................................................................................\n",
      "Total number of rows from Massachusetts is: ", len(customer_complaints_massachusetts), 
      "\n................................................................................\n")

# Filtering the complaints data to display only the rows containing a complaint and removing any empty/null rows

customer_complaints_massachusetts = customer_complaints_massachusetts[pd.notnull(
    customer_complaints_massachusetts['complaints_text_massachusetts'])]

number_of_non_null_rows = customer_complaints_massachusetts.shape[0]
print("The total number of rows after removing null entries is: ", str(number_of_non_null_rows), 
      "\n................................................................................\n")
print(customer_complaints_massachusetts[slice(5)])    # Displaying the first 5 rows for Massachusetts

................................................................................
 Total number of rows from Massachusetts is:  138 
................................................................................

The total number of rows after removing null entries is:  138 
................................................................................

                       complaints_text_massachusetts
0  Had Comcast. Overall the terrible experience e...
1  I was experiencing some technical difficulties...
2  Consistently, with the last couple of weeks (a...
3  Been dealing with a nightmare at Comcast for a...
4  I signed up for a one day pass with XFINITY wh...


---
## Implementing the Bag-of-words(BoW) technique using spaCy and sklearn
---

### Creating a list of complaints from the imported data

In [8]:
# Creating a list of complaints for each U.S. state respectively

arizona_complaints_list = customer_complaints_arizona.complaints_text_arizona.values.tolist()
massachusetts_complaints_list = customer_complaints_massachusetts.complaints_text_massachusetts.values.tolist()

# Printing first 5 complaints from Arizona
print("Arizona customer complaints: \n")
print(arizona_complaints_list[:5])

# Printing first 5 complaints from Massachusetts
print("\n"*2, "Massachusetts customer complaints: \n")
print(massachusetts_complaints_list[:5])

Arizona customer complaints: 

['First off, I\'ve been without internet for a week and on demand for a week as well. So I call the second day after nothing worked the first day to schedule a visit and the automated system kept saying that there was an outage in my area and it\'ll be resolved by 12:36. So I call later that afternoon and tell them and they try to send signals to the modem or whatever and it wouldn\'t work. So we scheduled a visit for that Friday between 10-12. Mind you that the first option they gave me was for the following freaking Monday and I said HECK NO that is way too long, I have a business to run.So the guy came on Friday at 1:30 NOT between 10-12 because I saw him pull up, he sat there for like 20 minutes and when I looked again he was freaking gone! So now I\'m a little hot, so I call and ask what the heck is going on and why\'d he just leave and the lady said, "Well there\'s another outage in your area and he wouldn\'t have been able to do anything but he sho

---
# Using spaCy
---

In [9]:
# Setting up the natural language processing/NLP pipeline

nlp = spacy.load("en_core_web_sm")    # Loading the "en_core_web_sm" language model due to its efficiency
stop_words = STOP_WORDS               # Storing default spaCy stopwords under variable name 'stop_words'
print("The stopwords are:  <",len(stop_words),">\n", stop_words, "\n\n") 
# Adding more words to the stopwords list
nlp.Defaults.stop_words |= {"gb", "heck", "oh", "wtf", "go", "will", "january", "february", "march", "april", "may", 
                            "june", "july", "august", "september", "october", "november", "december", "saturday", "sunday",
                            "monday", "tuesday", "wednesday", "thursday", "friday", "yr", "m", "mp","s","e","ma","o", "wt",
                            "non", "asap", "ss","dont", "gc", "etc"}
print("The updated stopwords are:  <",len(stop_words),">\n", stop_words, "\n\n")
stop_words_list = list(stop_words) # Creating a list of stopwords

# for word in stop_words:   # Printing stopwords and whether they are considered as stopwords
#     lexeme = nlp.vocab[word]
#     print(lexeme.text, lexeme.is_stop)

    
# Defining a function to remove 'unwanted' words from the processed text and add unwanted words to a list of removed words
 
def remove_unnecessary_words(main_list, stopwords, words_removed_list, list_of_processed_tokens, 
                             list_of_processed_text, list_of_lists_processed_text):
 
    for unwanted_word in main_list[:]:
        if unwanted_word in stopwords:
            print("The removed word is: ", unwanted_word, "\n\n")
            main_list.remove(unwanted_word)
            
            if unwanted_word not in words_removed_list:
                words_removed_list.append(unwanted_word)
            else:
                continue
        else:
            continue

    print("Updated list of length <", len(main_list),"> : ", main_list)  
    list_of_processed_tokens.append(main_list) # Processed complaint tokens are stored as a list in processed tokens list
#     print("List of processed tokens length: ", len(list_of_processed_tokens),"\n\n")
    joint_text_str = " ".join(main_list) # Creating a string by joining the processed tokens
    list_of_processed_text.append(joint_text_str) # String of processed tokens is added to list of processed text
    list_of_lists_processed_text.append([joint_text_str]) # String of processed tokens is stored as a list of lists


The stopwords are:  < 326 >
 {'else', 'hereupon', 'had', 'has', 'make', 'throughout', 'something', 'most', 'someone', 'many', 'would', 'ourselves', 'should', 'last', 'was', 'their', 'seems', 'from', 'using', 'hundred', 'off', 'doing', 'sometime', '’ve', 'that', 'becomes', 'around', 'move', 'so', 'front', 'herself', 'no', 'former', 'too', 'every', 'whenever', 'three', 'does', 'meanwhile', 'others', 'almost', 'whose', "'ll", 'any', 'noone', 'is', 'yet', 'to', "'ve", 'latterly', 'them', 'through', 'other', 'more', 'where', 'might', 'while', 'itself', 'indeed', 'out', '‘ve', 'six', 'yours', '’m', 'further', 'may', 'must', 'fifty', 'after', 'across', 'whereby', 'first', 'not', 'anyone', 'again', 'this', 'who', 'than', 'under', 'mine', '’s', 'between', '’ll', 'hers', 'it', 'put', 'third', 'never', 'for', 'along', 'name', 'by', 'only', 'amongst', 'back', 'ours', 'whoever', 'down', 'regarding', 'us', 'then', 'on', 'perhaps', 'some', 'being', 'forty', 'side', 'therein', 'whom', 'mostly', 'becau

<div style='text-align: left;'>
    <h3><i><u>
        Preprocessing complaints from Arizona
        </h3></i></u>
</div>  

In [10]:
start_time_spacy_ari = time.time()

list_of_complaints_a = []      # List of list of complaints, each element -> complaint as a list
plain_complaints_list_a = []   # List of complaints, each element -> complaint text
processed_tokens_list_a = []   # List of processed tokens in each text
vocabulary_a = []              # Empty list that will get appended with results i.e. unique words later in the code
words_removed_from_list_a = [] # List of words removed from original text having a size of <= 2

# Index is a variable that will be used to iterate through the complaints text list 
index_a = 0

# Using the while loop to iterate through the complaints_text_list
while len(arizona_complaints_list) > index_a:    
    for individual_complaint in arizona_complaints_list:
        # Removing emojis so that they don't get tokenized
        if len(emoji.emoji_list(individual_complaint)) > 0:
            print(emoji.emoji_list(individual_complaint), "\n"*2)
            arizona_complaints_list[index_a] =  emoji.replace_emoji(individual_complaint, replace="")
            print("The list has been upadated.\n\n", index_a, ": ", arizona_complaints_list[index_a], "\n"*2)
            individual_complaint = arizona_complaints_list[index_a]

        # Text Preprocessing
        doc_a = nlp(individual_complaint)
        # Lemmatizing tokens, converting them to lowercase and omitting stopwords, punctuations and numbers
        # processed_a creates string objects 
        processed_a = " ".join(token.lemma_.lower() for token in doc_a if not token.is_stop and not token.is_punct 
                               and token.is_alpha)
        print("\nprocessed_a ", index_a, " is: \n\n", processed_a,"\n\n")
        split_processed_a = processed_a.split()
        print("split_processed_a ", index_a, " is of length <", len(split_processed_a),"> :\n\n", split_processed_a,"\n\n")
        print("Removing unnecessary words ...\n\n")
        remove_unnecessary_words(split_processed_a, stop_words_list, words_removed_from_list_a, processed_tokens_list_a, 
                                 plain_complaints_list_a, list_of_complaints_a)
        print("\n\nUnnecessary words removed!\n\n")
        
        for word in split_processed_a:
            if word not in vocabulary_a:
                vocabulary_a.append(word)

        index_a += 1   # Incrementing index by 1
print("Vocabulary is of length  : <",len(vocabulary_a),">\n\n", vocabulary_a)
end_time_spacy_ari = time.time()
runtime_spacy_ari = round(end_time_spacy_ari - start_time_spacy_ari, 2)

print("\n\n ************************************************************************************************** \n\n", 
      "The time taken to run this code is: ", runtime_spacy_ari, "seconds", 
      "\n\n ************************************************************************************************** \n\n")

[{'match_start': 2022, 'match_end': 2023, 'emoji': '✌'}] 


The list has been upadated.

 0 :  First off, I've been without internet for a week and on demand for a week as well. So I call the second day after nothing worked the first day to schedule a visit and the automated system kept saying that there was an outage in my area and it'll be resolved by 12:36. So I call later that afternoon and tell them and they try to send signals to the modem or whatever and it wouldn't work. So we scheduled a visit for that Friday between 10-12. Mind you that the first option they gave me was for the following freaking Monday and I said HECK NO that is way too long, I have a business to run.So the guy came on Friday at 1:30 NOT between 10-12 because I saw him pull up, he sat there for like 20 minutes and when I looked again he was freaking gone! So now I'm a little hot, so I call and ask what the heck is going on and why'd he just leave and the lady said, "Well there's another outage in your area a


processed_a  3  is: 

 company want deal get comcast middle bill come extra charge person talk lie cable shut twice agree pay tell pay time talk tell different ask listen conversation will company crap unbelievable like say person work comcast lie look honest reputable company comcast comcast crooked company meet 


split_processed_a  3  is of length < 46 > :

 ['company', 'want', 'deal', 'get', 'comcast', 'middle', 'bill', 'come', 'extra', 'charge', 'person', 'talk', 'lie', 'cable', 'shut', 'twice', 'agree', 'pay', 'tell', 'pay', 'time', 'talk', 'tell', 'different', 'ask', 'listen', 'conversation', 'will', 'company', 'crap', 'unbelievable', 'like', 'say', 'person', 'work', 'comcast', 'lie', 'look', 'honest', 'reputable', 'company', 'comcast', 'comcast', 'crooked', 'company', 'meet'] 


Removing unnecessary words ...


The removed word is:  get 


The removed word is:  will 


The removed word is:  say 


Updated list of length < 43 > :  ['company', 'want', 'deal', 'comcast', 'middle'


processed_a  8  is: 

 decide comcast second chance believe well customer service know wrong order service headache place order online go ok receive welcome comcast email hour later receive email state need set installation order fully process call set appointment time hour call minute finally get installation date able access account online second tell package come dvr receive dvr box instead receive hd box hd tv spend hour chat representative dvr box order properly sit system limbo receive bill additional installation charge tell authorize tell install fee technician come install charge install fee additional outlet 


split_processed_a  8  is of length < 91 > :

 ['decide', 'comcast', 'second', 'chance', 'believe', 'well', 'customer', 'service', 'know', 'wrong', 'order', 'service', 'headache', 'place', 'order', 'online', 'go', 'ok', 'receive', 'welcome', 'comcast', 'email', 'hour', 'later', 'receive', 'email', 'state', 'need', 'set', 'installation', 'order', 'fully', 'process', 'ca


processed_a  11  is: 

 absolute bad experience comcast yesterday call simple transfer service guess simple young lady nice screw account transfer suppose service current location suspend yesterday time end concerned set account talk deadbeat baby daddy thirty minute maybe turn service day soon call pm cst issue resolve hell break loose talk shay tell turn service disconnect day transfer stupid right tell go turn place ask speak supervisor purposely transfer different department sale department hold minute andrew answer rude refuse transfer supervisor instead purposely transfer wrong department billing hold minute billing department mind balance account set auto pay account set auto pay cancel rep billing department speak understand english ask transfer supervisor refuse give access account refuse help hang pm cst resolve ask supervisor corey rude help put hold repeatedly tell want place hold help finally say forget cancel account disconnect service corey transfer customer solutions s


processed_a  14  is: 

 call internet cable service pay month bill equipment ship equipment date order service equipment receive short adapter box receive additional remote call ship adapter box immediately tiny box come package shipping charge adapter apply account adapter box include original box original shipping charge contact customer service time receive charge twice finally today fourth time time representative agree charge credit apply pm allen employee number inform credit bill shipping fee charge review supervisor explain tell twice past month ask speak supervisor place hold return say supervisor busy able discuss matter account specialist say ahead apply credit account phone explain past rep submit credit supervisor assure credit bill service interruption charge contact customer service shut service past go pay bill online pay month allow submit include pay contact account rep online chat discuss credit joancien type fully apply shipping fee credit account service restore p


processed_a  16  is: 

 contact comcast inquire service available area new area year tell representative let know future neighbor service ask phone internet big tv person tell yes service place address mention safe week walk verify decide bit time call speak matt inspection set give code tell check likely get let know call morning actually demand number give person speak todd believe say order give want set get number say check service today tomorrow call morning day force employee time hear order right place matt person talk actually note survey complete wait new address add system long week tell enter new address sorry different department okay suppose wait sadly wait phone line instal lose discount new home owner insurance call tell go tell wait week address enter week come house think reason delay get house place comcast hit water line trash line put equipment check builder course builder fight pay damage state tell week check address enter service send instal tell sorry tomorrow 

In [11]:
# Printing lists
print("\nlist_of_complaints_a: \n\n",list_of_complaints_a[:5], 
      "\n\nplain_complaints_list_a: \n\n", plain_complaints_list_a[:5], 
      "\n\nprocessed_tokens_list_a: \n\n",processed_tokens_list_a[:5],
      "\n\nvocabulary_a: \n\n", vocabulary_a,
      "\n\nwords_removed_from_list_a: \n\n", words_removed_from_list_a)


list_of_complaints_a: 

 [['internet week demand week second day work day schedule visit automate system outage area resolve later afternoon tell try send signal modem work schedule visit mind option follow freaking way long business run guy come pull sit like minute look freak little hot ask leave lady outage area able come door tell later reschedule appointment confirm church wait come like guy lady phone tell reschedule freaking like seriously check like min local office reschedule pick confirm like schedule confirm pick way come cancel freak yes correct piss rescheduled time like happen guarantee direct'], ['having comcast measly data cap year cancel transfer customer retention explain problem sign unlimited internet change unlimited limited like people stream netflix download video game know reason people internet wonderful customer retention specialist tell data cap free hbo year wow month high package tell high limit residential customer maybe change mind month problem second m

<div style='text-align: left;'>
    <h3><i><u>
        Preprocessing complaints from Massachusetts
        </h3></i></u>
</div>  

In [12]:
start_time_spacy_mas = time.time()

list_of_complaints_m = []      # List of list of complaints, each element -> complaint as a list
plain_complaints_list_m = []   # List of complaints, each element -> complaint text
processed_tokens_list_m = []   # List of processed tokens in each text
vocabulary_m = []              # Empty list that will get appended with results i.e. unique words later in the code
words_removed_from_list_m = [] # List of words removed from original text having a size of <= 2

# Index is a variable that will be used to iterate through the complaints text list 
index_m = 0

# Using the while loop to iterate through the complaints_text_list
while len(massachusetts_complaints_list) > index_m:    
    for individual_complaint in massachusetts_complaints_list:
        # Removing emojis so that they don't get tokenized
        if len(emoji.emoji_list(individual_complaint)) > 0:
            print(emoji.emoji_list(individual_complaint), "\n"*2)
            massachusetts_complaints_list[index_m] =  emoji.replace_emoji(individual_complaint, replace="")
            print("The list has been upadated.\n\n", index_m, ": ", massachusetts_complaints_list[index_m], "\n"*2)
            individual_complaint = massachusetts_complaints_list[index_m]

        # Text Preprocessing
        doc_m = nlp(individual_complaint)
        # Lemmatizing tokens, converting them to lowercase and omitting stopwords, punctuations and numbers
        # processed_m creates string objects 
        processed_m = " ".join(token.lemma_.lower() for token in doc_m if not token.is_stop and not token.is_punct 
                               and token.is_alpha)
        print("\nprocessed_m ", index_m, " is: \n\n", processed_m,"\n\n")
        split_processed_m = processed_m.split()
        print("split_processed_m ", index_m, " is of length <", len(split_processed_m),"> :\n\n", split_processed_m,"\n\n")
        print("Removing unnecessary words ...\n\n")
        remove_unnecessary_words(split_processed_m, stop_words_list, words_removed_from_list_m, processed_tokens_list_m, 
                                 plain_complaints_list_m, list_of_complaints_m)
        print("\n\nUnnecessary words removed!\n\n")
        
        for word in split_processed_m:
            if word not in vocabulary_m:
                vocabulary_m.append(word)

        index_m += 1   # Incrementing index by 1
print("Vocabulary is of length  : <",len(vocabulary_m),">\n\n", vocabulary_m)
end_time_spacy_mas = time.time()
runtime_spacy_mas = round(end_time_spacy_mas - start_time_spacy_mas, 2)

print("\n\n ************************************************************************************************** \n\n", 
      "The time taken to run this code is: ", runtime_spacy_mas, "seconds", 
      "\n\n ************************************************************************************************** \n\n")


processed_m  0  is: 

 comcast overall terrible experience describe provider area time deal ultimately move apartment area service provider call cancel account take forever finally hardware store receipt deal debt collector month claim owe money return hardware luckily physical proof honestly pay sure call deny liability kick different debt collection agency new month later finally go away use comcast good customer service accidentally disconnect hint want cancel service complain choose option customer service menu add service connect agent immediately ultimately connect right department directly probably disconnect time 


split_processed_m  0  is of length < 83 > :

 ['comcast', 'overall', 'terrible', 'experience', 'describe', 'provider', 'area', 'time', 'deal', 'ultimately', 'move', 'apartment', 'area', 'service', 'provider', 'call', 'cancel', 'account', 'take', 'forever', 'finally', 'hardware', 'store', 'receipt', 'deal', 'debt', 'collector', 'month', 'claim', 'owe', 'money', 'ret


processed_m  5  is: 

 plan get comcast internet provider cheat pricing package expensively horrible false advertisement claim fast internet provider imagine pay month average internet work thing like internet fluctuate like crazy slow phone internet service go customer support calling work tell darn thing time solution internet restart modem work try negotiate possibility fix issue curve direction try package make pay horrible service like go help go curve distract cheat pay request comcast take step create xfinity wifi modem consent start set create internet leave open people use internet pay single bill pay heavy load honestly new tech let explain false marketing company bullshit company modem internet box inside modern stuff inside new plastic case state art modem pay provide internet box work way fact pay rent modem bull crap like call help far disrespect hang midway answer question company provide internet life falsely advertise suggest look different internet service comcast we


processed_m  9  is: 

 good experience customer service long wait time helpful customer choice provider city bad company 


split_processed_m  9  is of length < 14 > :

 ['good', 'experience', 'customer', 'service', 'long', 'wait', 'time', 'helpful', 'customer', 'choice', 'provider', 'city', 'bad', 'company'] 


Removing unnecessary words ...


Updated list of length < 14 > :  ['good', 'experience', 'customer', 'service', 'long', 'wait', 'time', 'helpful', 'customer', 'choice', 'provider', 'city', 'bad', 'company']


Unnecessary words removed!



processed_m  10  is: 

 update comcast blast month ago wifi crawl say nicely way get promise speed dial fast course answer comcast soon figure service company sue false advertisement 


split_processed_m  10  is of length < 25 > :

 ['update', 'comcast', 'blast', 'month', 'ago', 'wifi', 'crawl', 'say', 'nicely', 'way', 'get', 'promise', 'speed', 'dial', 'fast', 'course', 'answer', 'comcast', 'soon', 'figure', 'service', 'company', 'sue', 'fal


processed_m  13  is: 

 fact product work properly bad start secondly terrible customer service ruin mood happen internet work properly call customer service absolutely wonderful treat respect come bad direct advanced wireless department whacko pick phone half awake answer phone dreadful hello professional sarcasm tell happen internet halfway hear guy yawn hang phone bad guy customer punishment whatsoever go comcast hire bad worker disappointed 


split_processed_m  13  is of length < 60 > :

 ['fact', 'product', 'work', 'properly', 'bad', 'start', 'secondly', 'terrible', 'customer', 'service', 'ruin', 'mood', 'happen', 'internet', 'work', 'properly', 'call', 'customer', 'service', 'absolutely', 'wonderful', 'treat', 'respect', 'come', 'bad', 'direct', 'advanced', 'wireless', 'department', 'whacko', 'pick', 'phone', 'half', 'awake', 'answer', 'phone', 'dreadful', 'hello', 'professional', 'sarcasm', 'tell', 'happen', 'internet', 'halfway', 'hear', 'guy', 'yawn', 'hang', 'phone', 'bad',


processed_m  19  is: 

 happen receive letter urge upgrade modem letter state fourth paragraph protect environment properly recycle old modem note unreturned equipment fee old modem late bill receive monthly charge modem call essentially tell bad return face monthly charge 


split_processed_m  19  is of length < 36 > :

 ['happen', 'receive', 'letter', 'urge', 'upgrade', 'modem', 'letter', 'state', 'fourth', 'paragraph', 'protect', 'environment', 'properly', 'recycle', 'old', 'modem', 'note', 'unreturned', 'equipment', 'fee', 'old', 'modem', 'late', 'bill', 'receive', 'monthly', 'charge', 'modem', 'call', 'essentially', 'tell', 'bad', 'return', 'face', 'monthly', 'charge'] 


Removing unnecessary words ...


The removed word is:  call 


Updated list of length < 35 > :  ['happen', 'receive', 'letter', 'urge', 'upgrade', 'modem', 'letter', 'state', 'fourth', 'paragraph', 'protect', 'environment', 'properly', 'recycle', 'old', 'modem', 'note', 'unreturned', 'equipment', 'fee', 'old', '

...


processed_m  111  is: 

 dispute upgrade order installation charge result phone customer service center charge installation fee order clear say waive tell send sign copy order proof order get copy get pen ask speak supervisor get friendly sure hold minute simply hang call tell previous minute hand successfully close issue resolve customer satisfaction base experience conclude comcast hire liar cheat send copy paper work 


split_processed_m  111  is of length < 60 > :

 ['dispute', 'upgrade', 'order', 'installation', 'charge', 'result', 'phone', 'customer', 'service', 'center', 'charge', 'installation', 'fee', 'order', 'clear', 'say', 'waive', 'tell', 'send', 'sign', 'copy', 'order', 'proof', 'order', 'get', 'copy', 'get', 'pen', 'ask', 'speak', 'supervisor', 'get', 'friendly', 'sure', 'hold', 'minute', 'simply', 'hang', 'call', 'tell', 'previous', 'minute', 'hand', 'successfully', 'close', 'issue', 'resolve', 'customer', 'satisfaction', 'base', 'experience', 'conclude', 'comcast', 'h


processed_m  120  is: 

 upgrade guide notification price change eliminate timer ability save dvr show vhs tape huge thing view program guide line hide new ad space keep bleep audio video setting customize view change view setting view read guide info 


split_processed_m  120  is of length < 37 > :

 ['upgrade', 'guide', 'notification', 'price', 'change', 'eliminate', 'timer', 'ability', 'save', 'dvr', 'show', 'vhs', 'tape', 'huge', 'thing', 'view', 'program', 'guide', 'line', 'hide', 'new', 'ad', 'space', 'keep', 'bleep', 'audio', 'video', 'setting', 'customize', 'view', 'change', 'view', 'setting', 'view', 'read', 'guide', 'info'] 


Removing unnecessary words ...


The removed word is:  show 


The removed word is:  keep 


Updated list of length < 35 > :  ['upgrade', 'guide', 'notification', 'price', 'change', 'eliminate', 'timer', 'ability', 'save', 'dvr', 'vhs', 'tape', 'huge', 'thing', 'view', 'program', 'guide', 'line', 'hide', 'new', 'ad', 'space', 'bleep', 'audio', 'video',


processed_m  125  is: 

 husband move new build house move tech comcast come set cable internet phone main road section lot house comcast line tap hook technician tell call comcast tell problem fix week worry take long service month husband call week transfer numerous time repeat situation help disclose party set line finally reach know talk ask direct line say continue hundred transfer repeat situation think document thing happen demand go tv week number screen answer unplug cable box plug wait half hour think fix give hbo month sorry automatically charge premium channel month instead notify night husband go internet fine go internet phone service work say strong connection access available husband call comcast talk kid novia scotia idea go ask service lose cable response think phone internet connect hate comcast laugh customer service commercial option area 


split_processed_m  125  is of length < 133 > :

 ['husband', 'move', 'new', 'build', 'house', 'move', 'tech', 'comcast', 'co


processed_m  128  is: 

 storm tv start switch cable box remain live condo bldg neighbor problem piece equipment home problem reset clock big clock surge protector call comcast suggest unplug cable signal source power wait hook send refresh signal work tv retry time luck bring cable box office exchange luck send tech want blame lightning tv believe tv break room lightning answer finally call get supervisor agree right thing file damage claim promise settle soon protocol send tech supervisor ok call hour window give call look couple week pass promise home call come bit pm try blame thing explain mention agree process claim tell expect check hour claim file bad tell claim lady fault know call insurance co settle icing cake review bill originally bundle include dvr pay second turn faulty box fry tv downgrade charge dvr call woman phone say geez louise bet face feel bad fio available area know automate back happy customer service important laugh cry cave offer tech sup broken tv examine w


processed_m  134  is: 

 harass comcast threaten internet disconnect supposedly internet usage shoot obscene number gig claim family use space hard drive contain tell sign comcast limit download record download illegal download pay download virus protection software update ask send writing completely deny say send stuff send customer download limit suppose know tell pay allow breach contract comcast fault use computer business purpose majority time disabled child computer dependent severe handwritten skill need computer comcast phone operator wish comply request right 


split_processed_m  134  is of length < 77 > :

 ['harass', 'comcast', 'threaten', 'internet', 'disconnect', 'supposedly', 'internet', 'usage', 'shoot', 'obscene', 'number', 'gig', 'claim', 'family', 'use', 'space', 'hard', 'drive', 'contain', 'tell', 'sign', 'comcast', 'limit', 'download', 'record', 'download', 'illegal', 'download', 'pay', 'download', 'virus', 'protection', 'software', 'update', 'ask', 'send', 'writi

In [13]:
# Printing lists
print("\nlist_of_complaints_m: \n\n",list_of_complaints_m[:5], 
      "\n\nplain_complaints_list_m: \n\n", plain_complaints_list_m[:5], 
      "\n\nprocessed_tokens_list_m: \n\n",processed_tokens_list_m[:5],
      "\n\nvocabulary_m: \n\n", vocabulary_m,
      "\n\nwords_removed_from_list_m: \n\n", words_removed_from_list_m)


list_of_complaints_m: 

 [['comcast overall terrible experience describe provider area time deal ultimately apartment area service provider cancel account forever finally hardware store receipt deal debt collector month claim owe money return hardware luckily physical proof honestly pay sure deny liability kick different debt collection agency new month later finally away use comcast good customer service accidentally disconnect hint want cancel service complain choose option customer service menu add service connect agent immediately ultimately connect right department directly probably disconnect time'], ['experience technical difficulty demand service technical support solution problem reset box couple attempt system run support rep ask able access movie yes set problem happen redo previous process tell set fiance entirely different issue automate system tell account balance think mistake speak billing dept find demand movie order month confuse lock service code need purchase code 

<div style='text-align: left;'>
    <h1><u>
        Creating BoW vectors
        </h1></u>
</div>  

---
## Arizona BoW vectors
---

In [14]:
# Unigram implementation - vocabulary used
s_time_ari = time.time()               
word_index_dict_a = {}
iterator_a = 0
vector_iterator_a = 0
for word in vocabulary_a:
    word_index_dict_a[iterator_a] = word
    iterator_a += 1
print("The word index dictionary is: \n\n" , word_index_dict_a, "\n"*2)

# enumerate -> to iterate through the list of reviews so that it is iterable
for iterator, complaint_text in enumerate(list_of_complaints_a):
    vector_a = np.zeros(len(vocabulary_a), dtype = int)
    bow_dictionary_a = defaultdict(int)
    
    for token_list in processed_tokens_list_a:
        for word in token_list:
            bow_dictionary_a[word] +=1

# Creating a list of values from BoW dictionary
bow_dict_a_values_list = list(bow_dictionary_a.values())
print("bow_dict_a_values_list: \n\n",bow_dict_a_values_list,"\n"*2)

# Appending the vector_a list to create BoW vectors using the bow_dict_a_values_list
while vector_iterator_a < len(vector_a):
    vector_a[vector_iterator_a] = bow_dict_a_values_list[vector_iterator_a]
    vector_iterator_a += 1
print("The bow dictionary is: \n\n", bow_dictionary_a,"\n"*2)
print("Vector a is: \n\n", vector_a,"\n")

e_time_ari = time.time()
t_time_ari = round(e_time_ari - s_time_ari, 2)
print("\n\n **************************************************************************************************\n\n",
      "Time taken to create BoW vectors = ", t_time_ari,
      " s\n\n ************************************************************************************************** \n\n")

The word index dictionary is: 

 {0: 'internet', 1: 'week', 2: 'demand', 3: 'second', 4: 'day', 5: 'work', 6: 'schedule', 7: 'visit', 8: 'automate', 9: 'system', 10: 'outage', 11: 'area', 12: 'resolve', 13: 'later', 14: 'afternoon', 15: 'tell', 16: 'try', 17: 'send', 18: 'signal', 19: 'modem', 20: 'mind', 21: 'option', 22: 'follow', 23: 'freaking', 24: 'way', 25: 'long', 26: 'business', 27: 'run', 28: 'guy', 29: 'come', 30: 'pull', 31: 'sit', 32: 'like', 33: 'minute', 34: 'look', 35: 'freak', 36: 'little', 37: 'hot', 38: 'ask', 39: 'leave', 40: 'lady', 41: 'able', 42: 'door', 43: 'reschedule', 44: 'appointment', 45: 'confirm', 46: 'church', 47: 'wait', 48: 'phone', 49: 'seriously', 50: 'check', 51: 'min', 52: 'local', 53: 'office', 54: 'pick', 55: 'cancel', 56: 'yes', 57: 'correct', 58: 'piss', 59: 'rescheduled', 60: 'time', 61: 'happen', 62: 'guarantee', 63: 'direct', 64: 'having', 65: 'comcast', 66: 'measly', 67: 'data', 68: 'cap', 69: 'year', 70: 'transfer', 71: 'customer', 72: 'ret

---
### Using sklearn for matrix representaion of BoW vectors (created using CountVectorizer)
---

In [15]:
# Using sklearn to create a matrix of word occurence frequency using bigrams

# Creating a CountVectorizer model having bigrams
model_a = CountVectorizer(analyzer = 'word', ngram_range = (2, 2), max_features = 50)

# Creating a data array from the plain complaints text data
data_array_a = model_a.fit_transform(plain_complaints_list_a).toarray()

# Creating a data frame/matrix with bigrams as columns and the complaints as rows where data is taken from 'data_array_a' 
# data array 
df_output_a = pd.DataFrame(data = data_array_a, columns = model_a.get_feature_names_out())

# Storing bigrams as a list
ngram_bow_list_ari = model_a.get_feature_names_out()

print(df_output_a.T, "\n")   # Printing the transpose of matrix 'df_output_a' to swap the rows with columns and vice versa
print(ngram_bow_list_ari)    # Printing bigrams

                      0   1   2   3   4   5   6   7   8   9   10  11  12  13  \
adapter box            0   0   0   0   0   0   0   0   0   0   0   0   0   0   
ask speak              0   0   0   0   0   0   0   0   0   0   1   1   0   0   
attorney general       0   0   0   0   0   0   0   1   0   0   0   0   0   0   
auto pay               0   0   0   0   0   0   0   0   0   0   0   2   0   0   
bill come              0   1   0   1   0   0   0   0   0   0   0   0   0   0   
bill pay               0   0   0   0   0   0   0   0   0   0   0   0   0   0   
billing department     0   0   0   0   0   0   0   0   0   0   0   2   0   0   
box leave              0   0   0   0   0   0   0   0   0   0   2   0   0   0   
cable company          0   0   0   0   0   0   0   0   0   1   0   0   1   0   
cable tv               0   0   0   0   0   0   0   1   0   0   0   0   1   0   
cancel service         0   0   0   0   0   0   0   0   0   0   1   0   0   2   
come house             0   0   0   0   1

---
## Massachusetts BoW vectors
---

In [16]:
# Unigram implementation - vocabulary used
s_time_mas = time.time()               
word_index_dict_m = {}
iterator_m = 0
vector_iterator_m = 0
for word in vocabulary_m:
    word_index_dict_m[iterator_m] = word
    iterator_m += 1
print("The word index dictionary is: \n\n" , word_index_dict_m, "\n"*2)

# enumerate -> to iterate through the list of reviews so that it is iterable
for iterator, complaint_text in enumerate(list_of_complaints_m): 
    vector_m = np.zeros(len(vocabulary_m), dtype = int)
    bow_dictionary_m = defaultdict(int)
    
    for token_list in processed_tokens_list_m:
        for word in token_list:
            bow_dictionary_m[word] +=1

# Creating a list of values from BoW dictionary
bow_dict_m_values_list = list(bow_dictionary_m.values())
print("bow_dict_m_values_list: \n\n",bow_dict_m_values_list,"\n"*2)

# Appending the vector_a list to create BoW vectors using the bow_dict_a_values_list
while vector_iterator_m < len(vector_m):
    vector_m[vector_iterator_m] = bow_dict_m_values_list[vector_iterator_m]
    vector_iterator_m += 1
print("The bow dictionary is: \n\n", bow_dictionary_m,"\n"*2)
print("Vector m is: \n\n", vector_m,"\n")

e_time_mas = time.time()
t_time_mas = round(e_time_mas - s_time_mas, 2)
print("\n\n **************************************************************************************************\n\n",
      "Time taken to create BoW vectors = ", t_time_mas,
      " s\n\n ************************************************************************************************** \n\n")

The word index dictionary is: 

 {0: 'comcast', 1: 'overall', 2: 'terrible', 3: 'experience', 4: 'describe', 5: 'provider', 6: 'area', 7: 'time', 8: 'deal', 9: 'ultimately', 10: 'apartment', 11: 'service', 12: 'cancel', 13: 'account', 14: 'forever', 15: 'finally', 16: 'hardware', 17: 'store', 18: 'receipt', 19: 'debt', 20: 'collector', 21: 'month', 22: 'claim', 23: 'owe', 24: 'money', 25: 'return', 26: 'luckily', 27: 'physical', 28: 'proof', 29: 'honestly', 30: 'pay', 31: 'sure', 32: 'deny', 33: 'liability', 34: 'kick', 35: 'different', 36: 'collection', 37: 'agency', 38: 'new', 39: 'later', 40: 'away', 41: 'use', 42: 'good', 43: 'customer', 44: 'accidentally', 45: 'disconnect', 46: 'hint', 47: 'want', 48: 'complain', 49: 'choose', 50: 'option', 51: 'menu', 52: 'add', 53: 'connect', 54: 'agent', 55: 'immediately', 56: 'right', 57: 'department', 58: 'directly', 59: 'probably', 60: 'technical', 61: 'difficulty', 62: 'demand', 63: 'support', 64: 'solution', 65: 'problem', 66: 'reset', 67:

bow_dict_m_values_list: 

 [307, 2, 8, 22, 1, 19, 25, 112, 31, 2, 12, 300, 38, 67, 3, 15, 3, 6, 3, 2, 1, 113, 13, 12, 19, 27, 1, 1, 2, 2, 129, 20, 3, 1, 2, 12, 6, 2, 64, 25, 10, 25, 21, 154, 1, 17, 1, 53, 14, 6, 12, 2, 16, 15, 9, 6, 24, 28, 3, 6, 7, 2, 12, 19, 3, 53, 8, 72, 5, 5, 9, 15, 44, 68, 8, 12, 6, 3, 29, 28, 1, 8, 9, 161, 3, 1, 32, 4, 5, 22, 9, 51, 18, 2, 30, 25, 2, 2, 4, 47, 7, 9, 37, 2, 2, 5, 16, 1, 21, 3, 5, 79, 6, 100, 16, 13, 6, 57, 31, 17, 10, 11, 1, 48, 2, 3, 1, 36, 1, 1, 2, 3, 3, 4, 15, 37, 16, 42, 73, 36, 2, 10, 6, 1, 5, 1, 2, 2, 21, 4, 8, 1, 1, 1, 11, 2, 33, 5, 2, 1, 1, 1, 14, 1, 1, 1, 9, 25, 2, 2, 8, 1, 2, 2, 3, 46, 19, 125, 47, 15, 9, 64, 34, 4, 2, 10, 1, 19, 5, 5, 1, 38, 15, 22, 6, 2, 2, 9, 55, 2, 15, 51, 25, 3, 23, 1, 9, 3, 2, 30, 1, 11, 4, 3, 5, 4, 2, 34, 1, 1, 1, 1, 3, 1, 1, 38, 2, 1, 1, 3, 5, 8, 1, 19, 19, 1, 29, 7, 102, 1, 2, 18, 22, 3, 55, 1, 2, 1, 2, 1, 4, 27, 1, 15, 21, 7, 5, 1, 3, 10, 2, 1, 20, 6, 6, 2, 8, 18, 7, 9, 2, 1, 10, 4, 13, 2, 43, 1, 6, 1, 12, 5, 2

---
### Using sklearn for matrix representaion of BoW vectors (created using CountVectorizer)
---

In [17]:
# Using sklearn to create a matrix of word occurence frequency using bigrams

# Creating a CountVectorizer model having bigrams
model_m = CountVectorizer(analyzer = 'word', ngram_range = (2, 2), max_features = 50)

# Creating a data array from the plain complaints text data
data_array_m = model_m.fit_transform(plain_complaints_list_m).toarray()

# Creating a data frame/matrix with bigrams as columns and the complaints as rows where data is taken from 'data_array_m' 
# data array 
df_output_m = pd.DataFrame(data = data_array_m, columns = model_m.get_feature_names_out())

# Storing bigrams as a list
ngram_bow_list_mas = model_m.get_feature_names_out()

print(df_output_m.T, "\n")   # Printing the transpose of matrix 'df_output_m' to swap the rows with columns and vice versa
print(ngram_bow_list_mas)    # Printing bigrams

                        0    1    2    3    4    5    6    7    8    9    ...  \
ask speak                 0    0    0    0    0    0    0    0    0    0  ...   
basic cable               0    0    0    0    0    0    0    0    0    0  ...   
bill comcast              0    0    0    0    0    0    0    0    0    0  ...   
bill pay                  0    0    0    0    0    1    0    0    0    0  ...   
cable box                 0    0    0    0    0    0    0    0    0    0  ...   
cable internet            0    0    0    0    0    0    0    0    0    0  ...   
cable service             0    0    0    0    0    0    0    0    0    0  ...   
cable tv                  0    0    0    0    0    0    0    0    0    0  ...   
cancel service            1    0    0    0    0    0    0    0    0    0  ...   
comcast customer          0    0    0    0    0    0    1    0    0    0  ...   
comcast send              0    0    0    0    0    0    0    0    0    0  ...   
comcast service           0 

<div style='text-align: left;'>
    <b><h3><u><i>
         Brief comparison of results - Text preprocessing and creation of BoW vectors  
        </b></h3></u></i>
</div>

<div style='font-family: Arial;'>
Text cleaning is an important preprocessing step as it allows a more consistent and accurate result at the end and avoids purposeless results (eg. if the most occuring word is 'the', it is not an important topic and does not indicate anything about the relevant topic in the complaints text). <b> spaCy </b> is used for preprocessing the complaints text.  Emojis had to be removed as a part of cleaning text so that they are not considered as separate tokens. Stopwords, punctuations and numeric values are also removed and the tokens are lemmatized (while considering the context in which lemmas are used) and turned to lowercase. <br>
    
    
The BoW vectors have been created in two ways while using n-grams: one having unigrams (n=1) which is created using while and for loops and the other having bigrams (n=2) created using sklearn's CountVectorizer which is also used to give a matrix representation of the results.<br>
    
    
The time taken for text preprocessing is given below (values given to 2 decimal places): 
    
    
<u>Arizona complaints</u> : <b>{{runtime_spacy_ari}} s</b>
    
    
<u>Massachusetts complaints</u> : <b>{{runtime_spacy_mas}} s</b>
    
    
Text preprocesisng was faster for <u>Arizona</u> than <u>Massachusetts</u> due to the smaller number of complaints.
    
    
The time taken to create <b>BoW vectors</b> is given below (values given to 2 decimal places): 

    
<u>Arizona BoW</u> : <b>{{t_time_ari}} s</b>

    
<u>Massachusetts BoW</u> : <b>{{t_time_mas}} s</b>

    
The creation of <b>BoW vectors</b> for <u>Arizona</u> was faster than that for <u>Massachusetts</u> as there were a fewer number of complaints from <u>Arizona</u> compared to those from <u>Massachusetts</u> (hence a fewer number of word frequency assignments had to be made, leading to a smaller matrix as compared to <u>Massachusetts</u>).   
</div>

<div style='text-align: left;'>
    <h1><u>
        Creating Term Frequency Inverse Document Frequency(TF-IDF) word vectors
        </h1></u>
</div> 

---
## TF-IDF for Arizona  
---

In [18]:
## TF-IDF using sklearn (while using unigrams and bigrams)
tfidf_s_time_ari= time.time()

tfidf_vectorizer_a = TfidfVectorizer(analyzer = "word", ngram_range=(1,2), min_df=0.1)
tfidf_fit_a = tfidf_vectorizer_a.fit_transform(plain_complaints_list_a)
tfidf_model_a = tfidf_vectorizer_a.fit_transform(plain_complaints_list_a).toarray()
feat_names_a = tfidf_vectorizer_a.get_feature_names_out() # Extracting feature names-> terms- unigrams and bigrams
data_a = pd.DataFrame(data = np.round(tfidf_model_a, 2), columns = feat_names_a)
print("Feature names are: \n", feat_names_a, "\n\n")

# # Printing tranpose matrix to represent bigrams as rows and complaint number as columns
print(data_a.T)  

tfidf_e_time_ari = time.time()
tfidf_t_time_ari = round(tfidf_e_time_ari - tfidf_s_time_ari, 2)
print("\n\n**************************************************************************************************** \n\n",
      "Time taken to create TF-TDF vectors = ", tfidf_t_time_ari, 
      " s\n\n**************************************************************************************************** \n\n")

Feature names are: 
 ['able' 'access' 'access account' 'account' 'add' 'additional' 'address'
 'advance' 'ago' 'agree' 'allow' 'answer' 'appointment' 'area' 'ask'
 'ask speak' 'assure' 'attorney' 'attorney general' 'automate' 'available'
 'bad' 'balance' 'believe' 'big' 'bill' 'bill come' 'bill credit'
 'bill equipment' 'box' 'business' 'cable' 'cable company' 'cable tv'
 'cancel' 'cancel service' 'care' 'catch' 'change' 'charge'
 'charge equipment' 'chat' 'check' 'claim' 'collect' 'comcast'
 'comcast service' 'comcast xfinity' 'come' 'come house' 'company'
 'company comcast' 'complaint' 'complete' 'concern' 'confirm' 'contact'
 'contract' 'conversation' 'correct' 'course' 'credit' 'credit bill'
 'current' 'customer' 'customer service' 'date' 'day' 'day day'
 'day technician' 'deal' 'deal comcast' 'decide' 'demand' 'department'
 'different' 'different department' 'direct' 'disconnect' 'discover'
 'dollar' 'door' 'employee' 'end' 'equipment' 'excuse' 'experience'
 'experience comcast' '

In [19]:
###   Setting n grams to be trigrams   ###
tfidf_s_time_aari= time.time()

vectorizer_a = TfidfVectorizer(ngram_range = (3,3), max_features = 15, norm="l2") 
model_a = vectorizer_a.fit_transform(plain_complaints_list_a).A  # Creating an array of the model
feature_names_a = vectorizer_a.get_feature_names_out() # Extracting feature names-> terms- trigrams
data_a_vectorizer_a = pd.DataFrame(data = np.round(model_a, 2), columns = vectorizer_a.get_feature_names_out())
# Printing tranpose matrix to represent bigrams as rows and complaint number as columns
print("Feature names are: \n", feature_names_a, "\n\n")
print(data_a_vectorizer_a.T)

tfidf_e_time_aari = time.time()
tfidf_t_time_aari = round(tfidf_e_time_aari - tfidf_s_time_aari, 2)
print("\n\n **************************************************************************************************** \n\n",
      "Time taken to create TF-TDF vectors = ", tfidf_t_time_ari, 
      " s\n\n **************************************************************************************************** ")

Feature names are: 
 ['account set auto' 'ask speak supervisor' 'attorney general office'
 'charge contact customer' 'come house leave' 'contact customer service'
 'extra equipment leave' 'pay month bill' 'previous rep submit'
 'problem come speak' 'problem disconnect talk' 'rep submit credit'
 'set auto pay' 'speak manager tell' 'state attorney general'] 


                           0    1    2    3    4    5    6     7    8    9   \
account set auto          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0   
ask speak supervisor      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0   
attorney general office   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.71  0.0  0.0   
charge contact customer   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0   
come house leave          0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.00  0.0  0.0   
contact customer service  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0   
extra equipment leave     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0   
pay mon

---
## TF-IDF for Massachusetts  
---

In [20]:
## TF-IDF using sklearn (while using unigrams and bigrams)
tfidf_s_time_mas= time.time()

tfidf_vectorizer_m = TfidfVectorizer(analyzer = "word", ngram_range=(1,2), min_df=0.1)
tfidf_fit_m = tfidf_vectorizer_m.fit_transform(plain_complaints_list_m)
tfidf_model_m = tfidf_vectorizer_m.fit_transform(plain_complaints_list_m).toarray()
feat_names_m = tfidf_vectorizer_m.get_feature_names_out() # Extracting feature names-> terms- unigrams and bigrams
data_m = pd.DataFrame(data = np.round(tfidf_model_m, 2), columns = feat_names_m)
print("Feature names are: \n", feat_names_m, "\n\n")

# # Printing tranpose matrix to represent bigrams as rows and complaint number as columns
print(data_m.T)  

tfidf_e_time_mas = time.time()
tfidf_t_time_mas = round(tfidf_e_time_mas - tfidf_s_time_mas, 2)
print("\n\n **************************************************************************************************** \n\n",
      "Time taken to create TF-TDF vectors = ", tfidf_t_time_mas, 
      " s\n\n **************************************************************************************************** \n\n")

Feature names are: 
 ['account' 'address' 'answer' 'area' 'ask' 'available' 'bad' 'bill' 'box'
 'business' 'cable' 'cable box' 'cancel' 'change' 'channel' 'charge'
 'comcast' 'comcast customer' 'come' 'company' 'contact' 'credit'
 'customer' 'customer service' 'day' 'deal' 'department' 'equipment'
 'experience' 'explain' 'fee' 'finally' 'find' 'fix' 'good' 'happen'
 'help' 'high' 'hold' 'home' 'hour' 'house' 'internet' 'internet service'
 'issue' 'know' 'later' 'leave' 'let' 'like' 'line' 'look' 'minute'
 'modem' 'money' 'month' 'need' 'new' 'number' 'order' 'package' 'pay'
 'pay month' 'people' 'person' 'phone' 'price' 'problem' 'provider'
 'receive' 'rep' 'representative' 'resolve' 'return' 'right' 'send'
 'service' 'set' 'sign' 'speak' 'spend' 'start' 'state' 'supervisor'
 'sure' 'switch' 'tech' 'technician' 'tell' 'thing' 'think' 'time' 'today'
 'try' 'tv' 'use' 'wait' 'want' 'watch' 'way' 'week' 'work' 'year'] 


                   0     1     2     3     4     5     6     7     8

In [21]:
###   Setting n grams to be trigrams   ###
tfidf_s_time_mmas= time.time()

vectorizer_m = TfidfVectorizer(ngram_range = (3,3), max_features = 15, norm="l2") 
model_m = vectorizer_m.fit_transform(plain_complaints_list_m).A # Creating an array of the model
feature_names_m = vectorizer_m.get_feature_names_out() # Extracting feature names-> terms- trigrams
data_m_vectorizer_m = pd.DataFrame(data = np.round(model_m, 2), columns = vectorizer_m.get_feature_names_out())
# Printing tranpose matrix to represent bigrams as rows and complaint number as columns
print("Feature names are: \n", feature_names_m, "\n\n")
print(data_m_vectorizer_m.T)

tfidf_e_time_mmas = time.time()
tfidf_t_time_mmas = round(tfidf_e_time_mmas - tfidf_s_time_mmas, 2)
print("\n\n **************************************************************************************************** \n\n",
      "Time taken to create TF-TDF vectors = ", tfidf_t_time_mmas, 
      " s\n\n **************************************************************************************************** ")

Feature names are: 
 ['ask speak supervisor' 'bad customer service' 'comcast customer service'
 'come house install' 'customer service experience' 'customer service rep'
 'customer service representative' 'local comcast store' 'new credit card'
 'pay month service' 'pay past bill' 'self install kit'
 'service customer service' 'service representative tell'
 'spend hour phone'] 


                                 0    1    2    3    4    5    6    7    8    \
ask speak supervisor             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
bad customer service             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
comcast customer service         0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   
come house install               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
customer service experience      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
customer service rep             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
customer service representative  0.0  0.0  0.0  0.0  0.0 

In [22]:
                                              ##  Comparison calculations  ##
# Ratio of how fast is BoW compared to TF-IDF in case of Arizona
ari_tf_bow_ratio = round(tfidf_t_time_ari/t_time_ari, 2)

# Ratio of how fast is the algorithm used in Massachusetts
mas_tf_bow_ratio = round(t_time_mas/tfidf_t_time_mas, 2)

<div style='text-align: left;'>
    <b><h3><u><i>
         Brief comparison of the results obtained from BoW and TF-IDF models 
        </b></h3></u></i>
</div>

In [52]:
md(f"The results of <b>TF-IDF</b> are better than <b>BoW</b> as the relevance of a term/word to the document/complaint is considered which is helpful in identifying the most important terms in the corpus(which is later helpful in finding the most prevalent topics in the corpus/sample data).<br><br>Unlike <b>TF-IDF</b>, context related information is not retained in the <b>BoW vector</b> as it only shows the word occurence frequency in a document/customer complaint which doesn't really contribute to the identification of relevance of the term to the document (although when <b>BoW vectors</b> are considered individually, the term with the highest occurring frequency is considered to be of highest importance which semantically, might not be the case).<br><br>Considering the results for <i>'customer service'</i> in <u>Arizona</u>,<br><br>1) The <b>BoW</b> and <b>TF-IDF</b> values for complaint 12 are '1' and '0.14' respectively.<br><br>2) The <b>BoW</b> and <b>TF-IDF</b> values for complaint 14 are '3' and '0.10' respectively.<br><br>Considering the results for <i>'internet service'</i> in <u>Massachusetts</u>,<br><br>1) The <b>BoW</b> and <b>TF-IDF</b> values for complaint 3 are '1' and '0.24' respectively.<br><br>2) The <b>BoW</b> and <b>TF-IDF</b> values for complaint 5 are '2' and '0.15' respectively.<br><br>As can be seen from the results above, the word frequency value of the <b>BoW vector</b> does not indicate its degree of relevance. Considering the term <i>'customer service'</i> in complaints from <u>Arizona</u>, it is more relevant to complaint 12 compared to complaint 14 despite of complaint 14 having a greater <b>BoW value</b> than complaint 12. Similarly, in the <u>Massachusetts</u> complaints text, the term <i>'internet service'</i> is more relevant to complaint 3 compared to complaint 5 despite of complaint 5 having a larger BoW value than complaint 3.<br><br><u>Note</u>: The complaint numbers are according to Python indexing, where the 1st element has an index of 0<br><br>The <b>BoW</b> and <b>TF-IDF</b> models of <u>Arizona</u> take <b>{t_time_ari} s</b> and <b>{tfidf_t_time_ari} s</b> respectively.<br><br>The <b>BoW</b> and <b>TF-IDF</b> models of <u>Massachusetts</u> take <b>{t_time_mas} s</b> and <b>{tfidf_t_time_mas} s</b> respectively.<br><br>Comparatively, the <b>BoW model</b> is about <b>{ari_tf_bow_ratio} times</b> faster than the <b>TF-IDF model</b> in case of <u>Arizona</u> but in case of <u>Massachusetts</u>, the <b>BoW model</b> is about <b>{mas_tf_bow_ratio} times</b> slower than the <b>TF-IDF model</b>.<br><br>For further analysis and processing, the TF-IDF model used (for both Arizona and Massachusetts) is the one that utilizes unigrams and bigrams.")


The results of <b>TF-IDF</b> are better than <b>BoW</b> as the relevance of a term/word to the document/complaint is considered which is helpful in identifying the most important terms in the corpus(which is later helpful in finding the most prevalent topics in the corpus/sample data).<br><br>Unlike <b>TF-IDF</b>, context related information is not retained in the <b>BoW vector</b> as it only shows the word occurence frequency in a document/customer complaint which doesn't really contribute to the identification of relevance of the term to the document (although when <b>BoW vectors</b> are considered individually, the term with the highest occurring frequency is considered to be of highest importance which semantically, might not be the case).<br><br>Considering the results for <i>'customer service'</i> in <u>Arizona</u>,<br><br>1) The <b>BoW</b> and <b>TF-IDF</b> values for complaint 12 are '1' and '0.14' respectively.<br><br>2) The <b>BoW</b> and <b>TF-IDF</b> values for complaint 14 are '3' and '0.10' respectively.<br><br>Considering the results for <i>'internet service'</i> in <u>Massachusetts</u>,<br><br>1) The <b>BoW</b> and <b>TF-IDF</b> values for complaint 3 are '1' and '0.24' respectively.<br><br>2) The <b>BoW</b> and <b>TF-IDF</b> values for complaint 5 are '2' and '0.15' respectively.<br><br>As can be seen from the results above, the word frequency value of the <b>BoW vector</b> does not indicate its degree of relevance. Considering the term <i>'customer service'</i> in complaints from <u>Arizona</u>, it is more relevant to complaint 12 compared to complaint 14 despite of complaint 14 having a greater <b>BoW value</b> than complaint 12. Similarly, in the <u>Massachusetts</u> complaints text, the term <i>'internet service'</i> is more relevant to complaint 3 compared to complaint 5 despite of complaint 5 having a larger BoW value than complaint 3.<br><br><u>Note</u>: The complaint numbers are according to Python indexing, where the 1st element has an index of 0<br><br>The <b>BoW</b> and <b>TF-IDF</b> models of <u>Arizona</u> take <b>0.02 s</b> and <b>0.25 s</b> respectively.<br><br>The <b>BoW</b> and <b>TF-IDF</b> models of <u>Massachusetts</u> take <b>0.52 s</b> and <b>0.2 s</b> respectively.<br><br>Comparatively, the <b>BoW model</b> is about <b>12.5 times</b> faster than the <b>TF-IDF model</b> in case of <u>Arizona</u> but in case of <u>Massachusetts</u>, the <b>BoW model</b> is about <b>2.6 times</b> slower than the <b>TF-IDF model</b>.<br><br>For further analysis and processing, the TF-IDF model used (for both Arizona and Massachusetts) is the one that utilizes unigrams and bigrams.

<div style='font-family: Arial;'>
The results of <b>TF-IDF</b> are better than <b>BoW</b> as the relevance of a term/word to the document/complaint is considered which is helpful in identifying the most important terms in the corpus(which is later helpful in finding the most prevalent topics in the corpus/sample data).  <br>
    
    
Unlike <b>TF-IDF</b>, context related information is not retained in the <b>BoW vector</b> as it only shows the word occurence frequency in a document/customer complaint which doesn't really contribute to the identification of relevance of the term to the document (although when <b>BoW vectors</b> are considered individually, the term with the highest occurring frequency is considered to be of highest importance which semantically, might not be the case).
    
        

Considering the results for <i>'customer service'</i> in <u>Arizona</u>,
    
1) The <b>BoW</b> and <b>TF-IDF</b> values for complaint 12 are '1' and '0.14' respectively.
    
2) The <b>BoW</b> and <b>TF-IDF</b> values for complaint 14 are '3' and '0.10' respectively.    
      
    
    
Considering the results for <i>'internet service'</i> in <u>Massachusetts</u>,
    
1) The <b>BoW</b> and <b>TF-IDF</b> values for complaint 3 are '1' and '0.24' respectively.
    
2) The <b>BoW</b> and <b>TF-IDF</b> values for complaint 5 are '2' and '0.15' respectively.
    
    
As can be seen from the results above, the word frequency value of the <b>BoW vector</b> does not indicate its degree of relevance. Considering the term <i>'customer service'</i> in complaints from <u>Arizona</u>, it is more relevant to complaint 12 compared to complaint 14 despite of complaint 14 having a greater <b>BoW value</b> than complaint 12. Similarly, in the <u>Massachusetts</u> complaints text, the term <i>'internet service'</i> is more relevant to complaint 3 compared to complaint 5 despite of complaint 5 having a larger BoW value than complaint 3.
    
    
<u>Note</u>: The complaint numbers are according to Python indexing, where the 1st element has an index of 0
    
    
The <b>BoW</b> and <b>TF-IDF</b> models of <u>Arizona</u> take <b>{{t_time_ari}} s</b> and <b>{{tfidf_t_time_ari}} s</b> respectively.

The <b>BoW</b> and <b>TF-IDF</b> models of <u>Massachusetts</u> take <b>{{t_time_mas}} s</b> and <b>{{tfidf_t_time_mas}} s</b> respectively.
    
    
Comparatively, the <b>BoW model</b> is about <b>{{ari_tf_bow_ratio}} times</b> faster than the <b>TF-IDF model</b> in case of <u>Arizona</u> but in case of <u>Massachusetts</u>, the <b>BoW model</b> is about <b>{{mas_tf_bow_ratio}} times</b> slower than the <b>TF-IDF model</b>. <br>
    
For further analysis and processing, the TF-IDF model used (for both Arizona and Massachusetts) is the one that utilizes unigrams and bigrams.
    

</div>

<div style='text-align: left;'>
    <h1><u>
        Topic extraction using different semantic analysis techniques 
        </h1></u>
</div> 


In [23]:
##  Defining a function to display top n words for each topic  ##
# The function arguments are:
  # components -< LDA/LSA components
  # n -> number of words to be displayed for each topic

def topic_displayer(components, n):
    n_words = n  # Variable to store number of words to be displayed

    for topic_id, topic_name in enumerate(components):
        top_n_words = feat_names_a[topic_name.argsort()[-n_words:][::-1]]
        top_features = ' '.join(top_n_words)
        print("Topic ",topic_id, " : ",top_features)

 <div style='text-align: left;'>
    <b><h2><u>
         Using Latent Semantic Analysis (LSA)
        </b></h2></u>
</div>

### LSA for Arizona

In [24]:
lsa_start_time_a = time.time()

# Setting number of topics to be 5 and number of iterations to be 500
lsa_model_a = TruncatedSVD(n_components = 5, algorithm ='randomized', random_state=50, n_iter=500)
lsa_a = lsa_model_a.fit_transform(tfidf_model_a)
lsa_components_a = lsa_model_a.components_
print("The results of LSA are:\n\n", lsa_a, "\n\n")
print("The LSA components are:\n\n", lsa_components_a, "\n\n")

counter_a = 0
complaints_list_length_a = len(plain_complaints_list_a)

while counter_a < complaints_list_length_a:
    l_a = lsa_a[counter_a]
    print(l_a, "\n")
    print("Complaint ", counter_a, " : ")

    for counter,topic in enumerate(l_a):
        print("Topic ",counter+1," : ",round(topic*100, 3),"%")
        counter += 1
    counter_a +=1

lsa_end_time_a = time.time()
lsa_total_time_a = round(lsa_end_time_a - lsa_start_time_a, 2) 
print("\n\n ****************************************************************************************************** \n\n",
      "Total time taken: ", lsa_total_time_a, 
      " s\n\n ****************************************************************************************************** \n\n")

The results of LSA are:

 [[ 0.39859846  0.59274744  0.03170351 -0.18434943  0.15595883]
 [ 0.44433995 -0.3479146   0.3437029  -0.30068193  0.05072858]
 [ 0.36506165  0.15548863 -0.02632745  0.51569845  0.09302873]
 [ 0.40632699 -0.01618082  0.06745908  0.23284707  0.50332167]
 [ 0.44579591  0.37421109 -0.16704077 -0.27916563  0.11557304]
 [ 0.28610036 -0.09836072  0.38014609  0.26451725  0.16378106]
 [ 0.3609915   0.48521867 -0.11811222 -0.17012643  0.01575923]
 [ 0.51654838 -0.12320423  0.48348303 -0.16291673 -0.1597934 ]
 [ 0.44695447  0.0069802  -0.24513643  0.19101819 -0.46019915]
 [ 0.47517318  0.24256902  0.30618523 -0.22965483 -0.34708362]
 [ 0.57947136 -0.07543095 -0.43854179 -0.24860252  0.12152632]
 [ 0.45139415  0.00381719 -0.0757168   0.39523964 -0.0696852 ]
 [ 0.37979174 -0.012006    0.37458906  0.09920264 -0.25610323]
 [ 0.35236216 -0.16618318 -0.29963019  0.12308666 -0.28295279]
 [ 0.59419831 -0.38417912 -0.33888955 -0.04897007 -0.15812452]
 [ 0.49243735 -0.45954095 -0.

In [25]:
# Printing top 10 words in each topic
topic_displayer(lsa_components_a, 10)

Topic  0  :  service bill comcast month tell come pay credit time charge
Topic  1  :  schedule come like reschedule confirm appointment sit day later work
Topic  2  :  problem phone month customer cable people internet year read require
Topic  3  :  transfer require company contact service help talk lie set person
Topic  4  :  bill pay company person lie like address meet work talk


### LSA for Massachusetts

In [26]:
lsa_start_time_m = time.time()

# Setting number of topics to be 5 and number of iterations to be 500
lsa_model_m = TruncatedSVD(n_components = 5, algorithm ='randomized', random_state=50, n_iter=500)
lsa_m = lsa_model_m.fit_transform(tfidf_model_m)
lsa_components_m = lsa_model_m.components_
print("The results of LSA are:\n\n", lsa_m, "\n\n")
print("The LSA components are:\n\n", lsa_components_m, "\n\n")

counter_m = 0
complaints_list_length_m = len(plain_complaints_list_m)

while counter_m < complaints_list_length_m:
    l_m = lsa_m[counter_m]
    print(l_m, "\n")
    print("Complaint ", counter_m, " : ")

    for counter,topic in enumerate(l_m):
        print("Topic ",counter+1," : ",round(topic*100, 3),"%")
        counter += 1
    counter_m +=1

lsa_end_time_m = time.time()
lsa_total_time_m = round(lsa_end_time_m - lsa_start_time_m, 2) 
print("\n\n ****************************************************************************************************** \n\n",
      "Total time taken: ", lsa_total_time_m, 
      " s\n\n ****************************************************************************************************** \n\n")

The results of LSA are:

 [[ 4.89429923e-01 -4.15370949e-02 -1.95554722e-01  6.41390027e-02
  -2.28404823e-01]
 [ 4.88179879e-01  6.50049259e-02 -1.42432501e-01 -2.43476466e-01
   1.51808085e-01]
 [ 2.93614639e-01 -2.63906976e-01 -2.52019634e-01  2.00765713e-02
  -2.36739094e-01]
 [ 4.83437980e-01  5.78186813e-03  2.36747542e-01  4.87817314e-01
   2.22024787e-01]
 [ 3.14195782e-01 -1.49556908e-01 -2.54009474e-01 -1.13664974e-01
   1.58311674e-02]
 [ 5.12130184e-01  2.15388271e-02  1.77322321e-01  5.58406941e-01
   2.10359003e-01]
 [ 5.10910398e-01  1.09779289e-01 -1.89519030e-01 -2.12022694e-02
  -1.61383306e-01]
 [ 4.35289533e-01  1.50029420e-01 -1.43412856e-01 -4.22241626e-02
   1.26557247e-01]
 [ 4.56923501e-01 -1.99157396e-02  7.98628409e-02  5.83947638e-01
   1.04606938e-01]
 [ 3.46523169e-01 -3.06707908e-01 -2.94425294e-01  3.06098080e-06
  -3.27790821e-01]
 [ 3.40321019e-01  8.90016063e-02 -6.09905311e-02  6.07044747e-02
  -1.91304960e-01]
 [ 6.74735354e-01 -1.07149473e-01 -1.60

In [27]:
# Printing top 10 words in each topic
topic_displayer(lsa_components_m, 10)

Topic  0  :  different department assure balance check explain credit allow confirm fee advance
Topic  1  :  advance credit confirm able business bad ask employee assure date
Topic  2  :  allow ask ago fix check answer date experience area credit
Topic  3  :  check claim credit different department customer service fix issue complete assure agree course
Topic  4  :  check ago complete explain different day technician bad direct big department


 <div style='text-align: left;'>
    <b><h2><u>
         Using Latent Dirichlet Allocation(LDA)
        </b></h2></u>
</div>

### LDA for Arizona

In [28]:
### Implementing LDA using sklearn
lda_start_time_a = time.time()

# Extracting 5 topics
lda_model_a = LatentDirichletAllocation(n_components=5, learning_method='online', random_state=50 ,max_iter=500)
lda_top_a = lda_model_a.fit_transform(tfidf_model_a)
lda_components_a = lda_model_a.components_
print("The LDA model is: \n",lda_model_a)
print("The results of LDA are: \n\n" , lda_top_a, "\n\n")
print("lda_components_a is: \n",lda_components_a, "\n\n")
lda_counter_a = 0

while lda_counter_a < complaints_list_length_a:
    print("Review ", lda_counter_a, " : \n")
    for i,topic in enumerate(lda_top_a[lda_counter_a]):
        print("Topic ",i+1,": ",round(topic*100,3),"%")
    print(" ")
    lda_counter_a += 1
    
lda_end_time_a = time.time()
lda_total_time_a = round(lda_end_time_a - lda_start_time_a, 2) 
print("\n\n ************************************************************************************************** \n\n",
      "Total time taken: ", lda_total_time_a, 
      "s\n\n ************************************************************************************************** \n\n")

The LDA model is: 
 LatentDirichletAllocation(learning_method='online', max_iter=500,
                          n_components=5, random_state=50)
The results of LDA are: 

 [[0.02739021 0.88860889 0.02865045 0.02782579 0.02752466]
 [0.03300747 0.03378756 0.0335744  0.86597421 0.03365637]
 [0.03095768 0.87532967 0.03120357 0.03134664 0.03116244]
 [0.03163743 0.03202856 0.87253842 0.03190953 0.03188606]
 [0.02740636 0.02789129 0.88941343 0.02772831 0.02756061]
 [0.03586212 0.0360426  0.03612344 0.8559793  0.03599254]
 [0.03063212 0.03103506 0.87675541 0.0308241  0.03075332]
 [0.02615922 0.02656633 0.02667861 0.89407534 0.02652051]
 [0.02830044 0.02860961 0.02907577 0.88513434 0.02887983]
 [0.03000053 0.03026946 0.87736065 0.03190802 0.03046133]
 [0.02490086 0.02571044 0.89819669 0.02563581 0.02555619]
 [0.02421967 0.02497577 0.02463268 0.0247306  0.90144128]
 [0.03245333 0.03291999 0.03300754 0.86904692 0.03257223]
 [0.03851544 0.84481649 0.03888756 0.03900921 0.0387713 ]
 [0.02481282 0.0

In [29]:
# Printing top 10 words in each topic
topic_displayer(lda_components_a, 10)

Topic  0  :  mind able fix issue cancel later like tv hour minute
Topic  1  :  day tell like service week cancel charge contact try address
Topic  2  :  come leave technician comcast schedule pay company tech work house
Topic  3  :  month customer people problem phone time internet year receive read
Topic  4  :  bill credit month account supervisor pay service transfer speak comcast


### LDA for Massachusetts

In [30]:
### Implementing LDA using sklearn
lda_start_time_m = time.time()

# Extracting 5 topics
lda_model_m = LatentDirichletAllocation(n_components=5, learning_method='online', random_state=50 ,max_iter=500)
lda_top_m = lda_model_m.fit_transform(tfidf_model_m)
lda_components_m = lda_model_m.components_
print("The LDA model is: \n",lda_model_m)
print("The results of LDA are: \n\n" , lda_top_m, "\n\n")
print("lda_components_m is: \n",lda_components_m, "\n\n")
lda_counter_m = 0

while lda_counter_m < complaints_list_length_m:
    print("Review ", lda_counter_m, " : \n")
    for i,topic in enumerate(lda_top_m[lda_counter_m]):
        print("Topic ",i+1,": ",round(topic*100,3),"%")
    print(" ")
    lda_counter_m += 1
    
lda_end_time_m = time.time()
lda_total_time_m = round(lda_end_time_m - lda_start_time_m, 2) 
print("\n\n ************************************************************************************************** \n\n",
      "Total time taken: ", lda_total_time_m, 
      "s\n\n ************************************************************************************************** \n\n")

The LDA model is: 
 LatentDirichletAllocation(learning_method='online', max_iter=500,
                          n_components=5, random_state=50)
The results of LDA are: 

 [[0.03633688 0.03637021 0.81791815 0.0363343  0.07304046]
 [0.0341078  0.03405799 0.86519564 0.0333849  0.03325367]
 [0.0637712  0.06380947 0.74474156 0.06402309 0.06365467]
 [0.0420173  0.04182007 0.76630851 0.0419721  0.10788202]
 [0.04675077 0.04649133 0.81216829 0.04638299 0.04820662]
 [0.03683634 0.03665998 0.85252042 0.036882   0.03710127]
 [0.0436799  0.0428499  0.82835695 0.0426184  0.04249485]
 [0.04355691 0.04303448 0.82654382 0.04323838 0.04362641]
 [0.04280145 0.04186467 0.83064971 0.04188856 0.04279562]
 [0.04952997 0.04951447 0.80089006 0.05078446 0.04928105]
 [0.06185211 0.06085435 0.75451152 0.06260062 0.0601814 ]
 [0.02904842 0.02925371 0.88280197 0.02985157 0.02904433]
 [0.03388483 0.03432384 0.86457064 0.03365162 0.03356908]
 [0.04711236 0.04770505 0.80934153 0.04792601 0.04791506]
 [0.34099073 0.0

In [31]:
# Printing top 10 words in each topic
topic_displayer(lda_components_m, 10)

Topic  0  :  confirm different department advance credit correct date business ask speak fix find
Topic  1  :  fix allow ago bill credit contact ask answer credit experience confirm
Topic  2  :  different department assure balance fee confirm allow explain ago check believe
Topic  3  :  allow experience comcast credit automate assure different department day additional ask come
Topic  4  :  department assure fix issue different explain check game credit disconnect agree


 <div style='text-align: left;'>
    <b><h2><u>
          Using Hierarchical Dirichlet Process (HDP)
        </b></h2></u>
</div>

In [32]:
# Defining a function to get the top 10 words in each topic extracted through HDP

def get_top_10_hdp_topic_words(hdp, n =10):
    # Getting most important topics by the number of times they were assigned i.e. counts
    sorted_topics = [key for key, value in sorted(enumerate(hdp.get_count_by_topics()), key = lambda x:x[1], reverse = True)]
    
    topics = dict()
    
    # For topics found, extract only those that are still assigned
    for key in sorted_topics:
        # To remove unassigned topics(those that aren't alive) at the end 
        if not hdp.is_live_topic(key):
            continue   
        topic_word_prob = []
        for word, prob in hdp.get_topic_words(key, n):
            topic_word_prob.append((word, prob))
            
        topics[key] = topic_word_prob   # Store array of topic word/frequency
        
    return topics
    

### HDP for Arizona

In [33]:
## This HDP code is inspired by 
## https://towardsdatascience.com/dont-be-afraid-of-nonparametric-topic-models-part-2-python-e5666db347a

# The HDP model is defined using the tomotopy library(extension of Topic Modelling Tool-->tomoto)
# Inverse Document Frequency term weighting is used to asssign higher weight to fewer occuring topics and vice versa
# Minimum collection frequnecy is set to 0 to prevent exclusion of words
# Minimum document frequency is set to 3 for exclusion of words that have a document frequency less than the set value
# Number of top words to be removed is set to 0 to keep the most frquently occuring words
# Initial k number of topics is set to 3 which is later adjusted during training
# alpha is set to 0.1 so that individual documents have a very few set of topics
# gamma is set to 1 so that documents share a lot of topics
# corpus is the list of docs to be added to model


start_time_hdp_ari_one = time.time()

hdp_model = tp.HDPModel(tw=tp.TermWeight.IDF,min_cf=0,min_df=3,rm_top=0,initial_k=3,alpha=0.1,gamma=1)

for token in processed_tokens_list_a:
    hdp_model.add_doc(token)

# Initiating Markov Chain Monte Carlo MCMC burn in to discard the first 7000 iterations and allow the model to stabilize 
# and converge i.e 'get into working condition'
hdp_model.burn_in = 7000
hdp_model.train(0)

print("No. of complaints: ", len(hdp_model.docs), "\nVocab size: ", hdp_model.num_vocabs,"\nNum words: ", 
      hdp_model.num_words)
print("Removed top words: ", hdp_model.removed_top_words)

end_time_hdp_ari_one = time.time()
total_time_hdp_ari_one = round(end_time_hdp_ari_one - start_time_hdp_ari_one, 2)
print("\n\n ************************************************************************************************** \n\n",
      "Total time taken: ", total_time_hdp_ari_one, 
      "s\n\n ************************************************************************************************** \n\n")

No. of complaints:  17 
Vocab size:  119 
Num words:  1077
Removed top words:  []


 ************************************************************************************************** 

 Total time taken:  0.01 s

 ************************************************************************************************** 




In [34]:
start_time_hdp_ari_two = time.time()
# Training the HDP model 
mcmc_iter = 5000
for i in range(0, mcmc_iter, 200):
    hdp_model.train(100, workers = 1000)
    print("Iteration: ",i,"    \tLog-likelihood: ",hdp_model.ll_per_word,"\tNo. of topics: ",hdp_model.live_k)    

end_time_hdp_ari_two = time.time()
total_time_hdp_ari_two = round(end_time_hdp_ari_two - start_time_hdp_ari_two, 2)
print("\n\n ************************************************************************************************** \n\n",
      "Total time taken: ", total_time_hdp_ari_two, 
      "s\n\n ************************************************************************************************** \n\n")

Iteration:  0     	Log-likelihood:  -5.405699721708385 	No. of topics:  28
Iteration:  200     	Log-likelihood:  -5.293708701499893 	No. of topics:  22
Iteration:  400     	Log-likelihood:  -5.244834816270354 	No. of topics:  19
Iteration:  600     	Log-likelihood:  -5.274324143012887 	No. of topics:  19
Iteration:  800     	Log-likelihood:  -5.245633790002286 	No. of topics:  18
Iteration:  1000     	Log-likelihood:  -5.259493883205508 	No. of topics:  22
Iteration:  1200     	Log-likelihood:  -5.354838089895071 	No. of topics:  22
Iteration:  1400     	Log-likelihood:  -5.193752688732441 	No. of topics:  22
Iteration:  1600     	Log-likelihood:  -5.206865315449023 	No. of topics:  18
Iteration:  1800     	Log-likelihood:  -5.271575231023014 	No. of topics:  17
Iteration:  2000     	Log-likelihood:  -5.276509009463845 	No. of topics:  21
Iteration:  2200     	Log-likelihood:  -5.284558271170303 	No. of topics:  20
Iteration:  2400     	Log-likelihood:  -5.2402764789176794 	No. of topi

In [35]:
hdp_topics_a = get_top_10_hdp_topic_words(hdp_model)
print(hdp_topics_a)

{3: [('rep', 0.07486492395401001), ('problem', 0.05990416184067726), ('technician', 0.057354509830474854), ('issue', 0.0509885773062706), ('phone', 0.04671125113964081), ('equipment', 0.04494340345263481), ('fix', 0.04428289830684662), ('customer', 0.04152828827500343), ('able', 0.037463024258613586), ('internet', 0.03692082315683365)], 28: [('ask', 0.1049078032374382), ('talk', 0.08503713458776474), ('right', 0.07282958179712296), ('lie', 0.07282958179712296), ('want', 0.06167111545801163), ('person', 0.06076478213071823), ('suppose', 0.05828044191002846), ('care', 0.04373130202293396), ('different', 0.04114203527569771), ('area', 0.03649241477251053)], 30: [('new', 0.10945363342761993), ('line', 0.0835534930229187), ('know', 0.0788353756070137), ('home', 0.07162710279226303), ('check', 0.06970762461423874), ('money', 0.05970070883631706), ('week', 0.0430324412882328), ('hear', 0.03986239433288574), ('house', 0.03986239433288574), ('year', 0.03667314350605011)], 2: [('credit', 0.25819

In [36]:
list_hdp_topics_a = list(hdp_topics_a.values()) # Creating a list of HDP topic values
print(list_hdp_topics_a, "\n"*2)

hdp_iter_a = 0
while len(list_hdp_topics_a) >hdp_iter_a:
    for topic_words in list_hdp_topics_a:
        print("Topic ", hdp_iter_a, " :\n\n", topic_words,"\n"*2)
        hdp_iter_a += 1

[[('rep', 0.07486492395401001), ('problem', 0.05990416184067726), ('technician', 0.057354509830474854), ('issue', 0.0509885773062706), ('phone', 0.04671125113964081), ('equipment', 0.04494340345263481), ('fix', 0.04428289830684662), ('customer', 0.04152828827500343), ('able', 0.037463024258613586), ('internet', 0.03692082315683365)], [('ask', 0.1049078032374382), ('talk', 0.08503713458776474), ('right', 0.07282958179712296), ('lie', 0.07282958179712296), ('want', 0.06167111545801163), ('person', 0.06076478213071823), ('suppose', 0.05828044191002846), ('care', 0.04373130202293396), ('different', 0.04114203527569771), ('area', 0.03649241477251053)], [('new', 0.10945363342761993), ('line', 0.0835534930229187), ('know', 0.0788353756070137), ('home', 0.07162710279226303), ('check', 0.06970762461423874), ('money', 0.05970070883631706), ('week', 0.0430324412882328), ('hear', 0.03986239433288574), ('house', 0.03986239433288574), ('year', 0.03667314350605011)], [('credit', 0.2581901252269745), 

### HDP for Massachusetts

In [37]:
## This HDP code is inspired by 
## https://towardsdatascience.com/dont-be-afraid-of-nonparametric-topic-models-part-2-python-e5666db347a

# The HDP model is defined using the tomotopy library(extension of Topic Modelling Tool-->tomoto)
# Inverse Document Frequency term weighting is used to asssign higher weight to fewer occuring topics and vice versa
# Minimum collection frequnecy is set to 0 to prevent exclusion of words
# Minimum document frequency is set to 9 for exclusion of words that have a document frequency less than the set value
# Number of top words to be removed is also set to 0 to keep the most frquently occuring words
# Initial k number of topics is set to 10 which is later adjusted during training
# alpha is set to 0.1 so that individual documents have a very few set of topics
# gamma is set to 1 so that documents share a lot of topics
# corpus is the list of docs to be added to model

start_time_hdp_mas_one = time.time()


hdp_model_m = tp.HDPModel(tw=tp.TermWeight.IDF,min_cf=0,min_df=9,rm_top=0,initial_k=10,alpha=0.1,gamma=1)

for token in processed_tokens_list_m:
    hdp_model_m.add_doc(token)

# Initiating Markov Chain Monte Carlo MCMC burn in to discard the first 7000 iterations and allow the model to stabilize 
# and converge i.e 'get into working condition'
hdp_model_m.burn_in = 7000
hdp_model_m.train(0)

print("No. of complaints: ", len(hdp_model_m.docs), "\nVocab size: ", hdp_model_m.num_vocabs,"\nNum words: ", 
      hdp_model_m.num_words)
print("Removed top words: ", hdp_model_m.removed_top_words)

end_time_hdp_mas_one = time.time()
total_time_hdp_mas_one = round(end_time_hdp_mas_one - start_time_hdp_mas_one, 2)
print("\n\n ************************************************************************************************** \n\n",
      "Total time taken: ", total_time_hdp_mas_one, 
      "s\n\n ************************************************************************************************** \n\n")

No. of complaints:  138 
Vocab size:  186 
Num words:  5941
Removed top words:  []


 ************************************************************************************************** 

 Total time taken:  0.02 s

 ************************************************************************************************** 




In [38]:
start_time_hdp_mas_two = time.time()

# Training the HDP model 
mcmc_iter_m = 5000
for i in range(0, mcmc_iter_m, 200):
    hdp_model_m.train(100, workers = 1000)
    print("Iteration: ",i,"    \tLog-likelihood: ",hdp_model_m.ll_per_word,"\tNo. of topics: ",hdp_model_m.live_k)    
    
end_time_hdp_mas_two = time.time()
total_time_hdp_mas_two = round(end_time_hdp_mas_two - start_time_hdp_mas_two, 2)
print("\n\n ************************************************************************************************** \n\n",
      "Total time taken: ", total_time_hdp_mas_two, 
      "s\n\n ************************************************************************************************** \n\n")

Iteration:  0     	Log-likelihood:  -5.254021236538034 	No. of topics:  8
Iteration:  200     	Log-likelihood:  -5.2479202409612045 	No. of topics:  7
Iteration:  400     	Log-likelihood:  -5.2480873934384125 	No. of topics:  7
Iteration:  600     	Log-likelihood:  -5.2438945156811325 	No. of topics:  7
Iteration:  800     	Log-likelihood:  -5.240580870130227 	No. of topics:  6
Iteration:  1000     	Log-likelihood:  -5.244742059123432 	No. of topics:  6
Iteration:  1200     	Log-likelihood:  -5.242960691303296 	No. of topics:  5
Iteration:  1400     	Log-likelihood:  -5.2437825367257895 	No. of topics:  5
Iteration:  1600     	Log-likelihood:  -5.246093205296465 	No. of topics:  5
Iteration:  1800     	Log-likelihood:  -5.243698305270445 	No. of topics:  5
Iteration:  2000     	Log-likelihood:  -5.245848374783559 	No. of topics:  5
Iteration:  2200     	Log-likelihood:  -5.264222954174914 	No. of topics:  4
Iteration:  2400     	Log-likelihood:  -5.246443662579165 	No. of topics:  4
It

In [39]:
hdp_topics_m = get_top_10_hdp_topic_words(hdp_model_m)
print(hdp_topics_m,"\n"*2)

{9: [('cable', 0.013167261146008968), ('bill', 0.012364540249109268), ('internet', 0.01235419325530529), ('box', 0.01161107886582613), ('tv', 0.011450272984802723), ('month', 0.010594792664051056), ('account', 0.010577376000583172), ('pay', 0.010254934430122375), ('payment', 0.00997130572795868), ('customer', 0.009890609420835972)], 8: [('appointment', 0.12082070112228394), ('schedule', 0.09454730898141861), ('technician', 0.07268828898668289), ('guy', 0.06754785776138306), ('department', 0.06589169800281525), ('half', 0.06043507903814316), ('hour', 0.060418836772441864), ('bad', 0.05651962757110596), ('wrong', 0.05404817312955856), ('early', 0.0467747338116169)], 6: [('mbps', 0.3518563210964203), ('speed', 0.30448177456855774), ('internet', 0.10973133146762848), ('technician', 0.07899696379899979), ('run', 0.0677739679813385), ('pay', 0.0318184494972229), ('order', 0.030118469148874283), ('new', 0.00014317316527012736), ('cancel', 0.00014316976012196392), ('phone', 0.00014315951557364

In [40]:
list_hdp_topics_m = list(hdp_topics_m.values()) # Creating a list of HDP topic values
print(list_hdp_topics_m, "\n"*2)

hdp_iter_m = 0
while len(list_hdp_topics_m) >hdp_iter_m:
    for topic_words in list_hdp_topics_m:
        print("Topic ", hdp_iter_m, " :\n\n", topic_words,"\n"*2)
        hdp_iter_m += 1

[[('cable', 0.013167261146008968), ('bill', 0.012364540249109268), ('internet', 0.01235419325530529), ('box', 0.01161107886582613), ('tv', 0.011450272984802723), ('month', 0.010594792664051056), ('account', 0.010577376000583172), ('pay', 0.010254934430122375), ('payment', 0.00997130572795868), ('customer', 0.009890609420835972)], [('appointment', 0.12082070112228394), ('schedule', 0.09454730898141861), ('technician', 0.07268828898668289), ('guy', 0.06754785776138306), ('department', 0.06589169800281525), ('half', 0.06043507903814316), ('hour', 0.060418836772441864), ('bad', 0.05651962757110596), ('wrong', 0.05404817312955856), ('early', 0.0467747338116169)], [('mbps', 0.3518563210964203), ('speed', 0.30448177456855774), ('internet', 0.10973133146762848), ('technician', 0.07899696379899979), ('run', 0.0677739679813385), ('pay', 0.0318184494972229), ('order', 0.030118469148874283), ('new', 0.00014317316527012736), ('cancel', 0.00014316976012196392), ('phone', 0.00014315951557364315)]] 



In [41]:
                                              ##  Comparison calculations  ##
##  Total time taken for hdp model to run  ##

# Arizona hdp total time
ari_hdp_total_time = round(total_time_hdp_ari_one + total_time_hdp_ari_two,2)

# Massachusetts hdp total time
mas_hdp_total_time = round(total_time_hdp_mas_one + total_time_hdp_mas_two,2)


##   LDA/LSA ratio calulation for finding which technique is faster  ##

# Arizona lda/lsa
ari_lda_lsa_ratio = round(lda_total_time_a/lsa_total_time_a, 2)

# Massachusetts lda/lsa
mas_lda_lsa_ratio = round(lda_total_time_m/lsa_total_time_m, 2)


##  LDA/HDP ratio calulation for finding which technique is faster  ##

# Arizona lda/lsa
ari_lda_hdp_ratio = round(lda_total_time_a/ari_hdp_total_time, 2)

# Massachusetts lda/lsa
mas_lda_hdp_ratio = round(lda_total_time_m/mas_hdp_total_time, 2)


##  HDP/LSA ratio calulation for finding which technique is faster  ##

# Arizona hdp/lsa
ari_hdp_lsa_ratio = round(ari_hdp_total_time/lsa_total_time_a, 2)

# Massachusetts hdp/lsa
mas_hdp_lsa_ratio = round(mas_hdp_total_time/lsa_total_time_m, 2)


md(f"The LSA and LDA models of Arizona take {lsa_total_time_a} s and {lda_total_time_a} s respectively.\n")
md(f"The LSA and LDA models of Massachusetts take {lsa_total_time_m} s and {lda_total_time_m} s respectively.")
md(f"Comparatively, the LSA model is about {ari_lda_lsa_ratio} times faster than the LDA model in case of Arizona and about {mas_lda_lsa_ratio} times faster in case of Massachusetts.)

The HDP model of Arizona takes 1.7 s.

The HDP model of Massachusetts takes 7.16 s.


<div style='text-align: left;'>
    <b><h3><u><i>
        Brief comparison of results from LSA, LDA and HDP
        </b></h3></u></i>
</div>

In [51]:
md(f"<b>Latent Semantic Analysis (LSA)</b> gives a score for the most relevant topic whereas <b>Latent Dirichlet Allocation (LDA)</b> gives the relevance of individual complaints to each topic as a percentage.<br><br>The __LSA__ and __LDA__ models of <u>Arizona</u> take <b>{lsa_total_time_a} s</b> and <b>{lda_total_time_a} s</b> respectively.<br><br>The <b>LSA</b> and <b>LDA</b> models of <u>Massachusetts</u> take <b>{lsa_total_time_m} s</b> and <b>{lda_total_time_m} s</b> respectively.<br><br>Comparatively, the <b>LSA</b> model is about <b>{ari_lda_lsa_ratio} times</b> faster than the <b>LDA</b> model in case of <u>Arizona</u> and about <b>{mas_lda_lsa_ratio} times</b> faster in case of <u>Massachusetts</u>.<br><br>The <b>HDP</b> model of <u>Arizona</u> takes <b>{ari_hdp_total_time} s</b>.<br><br>The <b>HDP model</b> of <u>Massachusetts</u> takes <b>{mas_hdp_total_time} s</b>.<br><br><b>Hierarchical Dirichlet Processing (HDP)</b> is a topic extraction technique that is similar to <b>LDA</b> (uses Dirichlet distribution) but instead of explictly requiring the specification of the number of topics to be extracted from text, this model interprets the number of topics on its own based on the input corpus. It does take time to train the model but comparatively, the total runtime of the <b>HDP</b> model is less than that of <b>LDA</b>.<br><br>The <b>HDP</b> model is <b>{ari_lda_hdp_ratio} times</b> faster than <b>LDA</b> in case of <u>Arizona</u> and <b>{mas_lda_hdp_ratio} times</b> faster in case of <u>Massachusetts</u>.<br><br>The <b>LSA</b> model is <b>{ari_hdp_lsa_ratio} times</b> faster than <b>HDP</b> in case of <u>Arizona</u> and <b>{mas_hdp_lsa_ratio} times</b> faster in case of <u>Massachusetts</u>.<br><br>According to the results obtained above, <b>LSA</b> is faster than <b>HDP</b> which is faster than <b>LDA</b>. Although <b>LSA</b> is the fastest technique to use, it is also less accurate compared to the rest of the two techniques. <b>LDA</b> is more accurate than <b>LSA</b> but is also the slowest compared to the other two. <b>HDP</b> on the other hand, is faster than <b>LDA</b> but slower than <b>LSA</b> and is more accurate than <b>LSA</b>.<br><br><u>Note:</u> Both LDA and LSA require trial and error (or using coherence value/score) to find out the optimal number of topics extracted from the text. No trial and error is involved in the HDP method as multiple iterations are used to train the model which then finds out the optimal number of topics that the input text contains.<br><br>Overall, proper preprocessing of input text (ensuring no stopwords remain in the text), the selection of vectorization technique (like BoW and TF-IDF) and topic extraction technique (like LSA, LDA and HDP as some techniques are more efficient than the other whereas some are more accurate) has an impact on results and making the right choices at the beginning helps get more accurate/better results.<br><br>")


<b>Latent Semantic Analysis (LSA)</b> gives a score for the most relevant topic whereas <b>Latent Dirichlet Allocation (LDA)</b> gives the relevance of individual complaints to each topic as a percentage.<br><br>The __LSA__ and __LDA__ models of <u>Arizona</u> take <b>0.31 s</b> and <b>2.98 s</b> respectively.<br><br>The <b>LSA</b> and <b>LDA</b> models of <u>Massachusetts</u> take <b>0.33 s</b> and <b>24.5 s</b> respectively.<br><br>Comparatively, the <b>LSA</b> model is about <b>9.61 times</b> faster than the <b>LDA</b> model in case of <u>Arizona</u> and about <b>74.24 times</b> faster in case of <u>Massachusetts</u>.<br><br>The <b>HDP</b> model of <u>Arizona</u> takes <b>1.7 s</b>.<br><br>The <b>HDP model</b> of <u>Massachusetts</u> takes <b>7.16 s</b>.<br><br><b>Hierarchical Dirichlet Processing (HDP)</b> is a topic extraction technique that is similar to <b>LDA</b> (uses Dirichlet distribution) but instead of explictly requiring the specification of the number of topics to be extracted from text, this model interprets the number of topics on its own based on the input corpus. It does take time to train the model but comparatively, the total runtime of the <b>HDP</b> model is less than that of <b>LDA</b>.<br><br>The <b>HDP</b> model is <b>1.75 times</b> faster than <b>LDA</b> in case of <u>Arizona</u> and <b>3.42 times</b> faster in case of <u>Massachusetts</u>.<br><br>The <b>LSA</b> model is <b>5.48 times</b> faster than <b>HDP</b> in case of <u>Arizona</u> and <b>21.7 times</b> faster in case of <u>Massachusetts</u>.<br><br>According to the results obtained above, <b>LSA</b> is faster than <b>HDP</b> which is faster than <b>LDA</b>. Although <b>LSA</b> is the fastest technique to use, it is also less accurate compared to the rest of the two techniques. <b>LDA</b> is more accurate than <b>LSA</b> but is also the slowest compared to the other two. <b>HDP</b> on the other hand, is faster than <b>LDA</b> but slower than <b>LSA</b> and is more accurate than <b>LSA</b>.<br><br><u>Note:</u> Both LDA and LSA require trial and error (or using coherence value/score) to find out the optimal number of topics extracted from the text. No trial and error is involved in the HDP method as multiple iterations are used to train the model which then finds out the optimal number of topics that the input text contains.<br><br>Overall, proper preprocessing of input text (ensuring no stopwords remain in the text), the selection of vectorization technique (like BoW and TF-IDF) and topic extraction technique (like LSA, LDA and HDP as some techniques are more efficient than the other whereas some are more accurate) has an impact on results and making the right choices at the beginning helps get more accurate/better results.<br><br>

<div style='font-family: Arial;'>
<b>Latent Semantic Analysis (LSA)</b> gives a score for the most relevant topic whereas <b>Latent Dirichlet Allocation (LDA)</b> gives the relevance of individual complaints to each topic as a percentage.  <br>
                   
                  
The <b>LSA</b> and <b>LDA</b> models of <u>Arizona</u> take <b>{{lsa_total_time_a}} s</b> and <b>{{lda_total_time_a}} s</b> respectively.
    
The <b>LSA</b> and <b>LDA</b> models of <u>Massachusetts</u> take <b>{{lsa_total_time_m}} s</b> and <b>{{lda_total_time_m}} s</b> respectively.
    
    
Comparatively, the <b>LSA</b> model is about <b>{{ari_lda_lsa_ratio}} times</b> faster than the <b>LDA</b> model in case of <u>Arizona</u> and about <b>{{mas_lda_lsa_ratio}} times</b> faster in case of <u>Massachusetts</u>.
    
    
    
The <b>HDP</b> model of <u>Arizona</u> takes <b>{{ari_hdp_total_time}} s</b>.
    
The <b>HDP model</b> of <u>Massachusetts</u> takes <b>{{mas_hdp_total_time}} s</b>.    
    

<b>Hierarchical Dirichlet Processing (HDP)</b> is a topic extraction technique that is similar to <b>LDA</b> (uses Dirichlet distribution) but instead of explictly requiring the specification of the number of topics to be extracted from text, this model interprets the number of topics on its own based on the input corpus. It does take time to train the model but comparatively, the total runtime of the <b>HDP</b> model is less than that of <b>LDA</b>. 
    
    
The <b>HDP</b> model is <b>{{ari_lda_hdp_ratio}} times</b> faster than <b>LDA</b> in case of <u>Arizona</u> and <b>{{mas_lda_hdp_ratio}} times</b> faster in case of <u>Massachusetts</u>.
    
    
The <b>LSA</b> model is <b>{{ari_hdp_lsa_ratio}} times</b> faster than <b>HDP</b> in case of <u>Arizona</u> and <b>{{mas_hdp_lsa_ratio}} times</b> faster in case of <u>Massachusetts</u>. <br>
    
    
According to the results obtained above, <b>LSA</b> is faster than <b>HDP</b> which is faster than <b>LDA</b>. Although <b>LSA</b> is the fastest technique to use, it is also less accurate compared to the rest of the two techniques. <b>LDA</b> is more accurate than <b>LSA</b> but is also the slowest compared to the other two. <b>HDP</b> on the other hand, is faster than <b>LDA</b> but slower than <b>LSA</b> and is more accurate than <b>LSA</b>. <br>
    
<u>Note:</u> Both LDA and LSA require trial and error (or using coherence value/score) to find out the optimal number of topics extracted from the text. No trial and error is involved in the HDP method as multiple iterations are used to train the model which then finds out the optimal number of topics that the input text contains. <br> 
    
    
Overall, proper preprocessing of input text (ensuring no stopwords remain in the text), the selection of vectorization technique (like BoW and TF-IDF) and topic extraction technique (like LSA, LDA and HDP as some techniques are more efficient than the other whereas some are more accurate) has an impact on results and making the right choices at the beginning helps get more accurate/better results.
</div>